In [1]:
## Victorian400 데이터셋 다운 후 압축 해제

!git clone https://github.com/Pseudo-Lab/Tutorial-Book-Utils
!python Tutorial-Book-Utils/PL_data_loader.py --data GAN-Colorization
!unzip -q Victorian400-GAN-colorization-data.zip

Cloning into 'Tutorial-Book-Utils'...
remote: Enumerating objects: 45, done.
remote: Counting objects: 100% (45/45), done.
remote: Compressing objects: 100% (39/39), done.
remote: Total 45 (delta 18), reused 17 (delta 5), pack-reused 0
Receiving objects: 100% (45/45), 11.62 KiB | 11.62 MiB/s, done.
Resolving deltas: 100% (18/18), done.
Downloading...
From: https://drive.google.com/uc?id=1dZxoBIWmbuF-Oy_XZoS1z9EjPJwkTmy6
To: /content/Victorian400-GAN-colorization-data.zip
100% 508M/508M [00:02<00:00, 192MB/s]
Victorian400-GAN-colorization-data.zip download complete!
unzip:  cannot find or open Victorian400-GAN-colorization-data.zip}, Victorian400-GAN-colorization-data.zip}.zip or Victorian400-GAN-colorization-data.zip}.ZIP.


In [2]:
import os
import glob
# 위에까지 경로지정.

import datetime
import matplotlib.pyplot as plt
from PIL import Image # 이미지 파일 시각화.
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torch.autograd import Variable # 미분하는 거. 요새는 그냥 내장되어 있어서 굳이 안씀.
from torch.utils.data import Dataset, DataLoader

In [3]:
class VictorianDataset(Dataset):
  def __init__(self, root, color_transforms_ = None, gray_transforms_ = None):

    self.color_transforms = transforms.Compose(color_transforms_)
    self.gray_transforms = transforms.Compose(gray_transforms_)
    self.gray_files = sorted(glob.glob(os.path.join('/content/gray') + '/*.*'))
    self.color_files = sorted(glob.glob(os.path.join('/content/resized') + '/*.*'))


  def __getitem__(self, index):

    gray_img = Image.open(self.gray_files[index % len(self.gray_files)]).convert('L') # 흑백 이미지 convert로 1채널로 받아주고,
    color_img = Image.open(self.color_files[index % len(self.color_files)]).convert('RGB') # 컬러는 3채널 값으로.


    gray_img = self.gray_transforms(gray_img)
    color_img = self.color_transforms(color_img)


    return {'A' : gray_img, 'B': color_img}

  def __len__(self):

    return len(self.gray_files)

In [4]:
root = ''
test_root = root + 'test/'

img_height = 256
img_width = 256

batch_size = 12
test_batch_size = 6

gpu = 0

In [5]:
### 이전 코드 실습값 갖고 온거.
color_mean = [0.58090717, 0.52688643, 0.45678478]
color_std = [0.25644188, 0.25482641, 0.24456465]
gray_mean = [0.5350533]
gray_std = [0.25051587]


color_transforms_ = [
    transforms.Resize(size = (img_height, img_width)),
    transforms.ToTensor(),
    transforms.Normalize(mean = color_mean, std = color_std),
]

gray_transforms_ = [
    transforms.Resize(size = (img_height, img_width)),
    transforms.ToTensor(),
    transforms.Normalize(mean = gray_mean, std = gray_std),
]

In [6]:
### train 로더

train_loader = DataLoader(
    VictorianDataset(root, color_transforms_ = color_transforms_, gray_transforms_ = gray_transforms_),
    batch_size = batch_size,
    shuffle = True
)

ValueError: ignored

In [ ]:
### 데이터 잘 되어있는지 확인 작업.

def reNormalize(img, mean, std):
  img = img.numpy().transpose(1, 2, 0)
  img = img * std + mean
  img = img.clip(0, 1)

  return img

In [ ]:
fig = plt.figure(figsize = (10, 5))
rows = 1
cols = 2

for X in train_loader:

  print(X['A'].shape, X['B'].shape)

  ax1 = fig.add_subplot(rows, cols, 1)
  ax1.imshow(reNormalize(X['A'][0], gray_mean, gray_std).reshape(img_height, img_width), cmap = 'gray')
  ax1.set_title('gray img')

  ax2 = fig.add_subplot(rows, cols, 2)
  ax2.imshow(reNormalize(X['B'][0], color_mean, color_std))
  ax2.set_title('color img')

  plt.show()

  break

In [ ]:
### test 로더

test_loader = DataLoader(
    VictorianDataset(test_root, color_transforms_= color_transforms_, gray_transforms_ = gray_transforms_),
    batch_size = test_batch_size,
    shuffle = False
)

In [ ]:
fig = plt.figure(figsize = (10, 5))
rows = 1
cols = 2

for X in test_loader:

  print(X['A'].shape, X['B'].shape)
  ax1 = fig.add_subplot(rows, cols, 1)
  ax1.imshow(reNormalize(X['A'][0], gray_mean, gray_std).reshape(img_height, img_width), cmap = 'gray')
  ax1.set_title('gray img')

  ax2 = fig.add_subplot(rows, cols, 2)
  ax2.imshow(reNormalize(X['B'][0], color_mean, color_std))
  ax2.set_title('color img')

  plt.show()

  break

In [ ]:
##### GAN 모델 구축.

In [ ]:
class Generator(nn.Module):
  def __init__(self):

    super(Generator, self).__init__()

    self.conv1 = nn.Sequential(
        nn.Conv2d(in_channels= 2, out_channels = 64, kernel_size = 3, stride = 1, padding = 1),
        nn.BatchNorm2d(64), # 출력 채널에 맞춰서.
        nn.LeakyReLU(0.1), # LeakyReLU가 입력값이 0보다 크면, 그대로, 0보다 작음 0.1 곱해서. 그래프 떠올려보자.
    )

    self.maxpool = nn.MaxPool2d(2, 2)


    self.conv2 = nn.Sequential(
        nn.Conv2d(in_channels = 64, out_channels = 64 * 2, kernel_size = 3, stride = 1, padding = 1),
        nn.BatchNorm2d(64),
        nn.LeakyReLU(0.1)
    )

    self.upsample = nn.Sequential(
        nn.ConvTranspose2d(in_channels = 64 * 2, out_channels = 64, kernel_size = 4, stride = 2, padding = 1),
        nn.BatchNorm2d(64),
        nn.LeakyReLU(0.2)
    )

    self.conv1by1 = nn.Sequential(
        nn.Conv2d(in_channels = 64, out_channels = 64, kernel_size = 1, stride = 1, padding = 0),
        nn.BatchNorm2d(64),
        nn.LeakyReLU(0.1)
    )

    self.conv = nn.Sequential(
        nn.Conv2d(in_channels = 64, out_channels = 3, kernel_size = 3, stride = 1, padding = 1),
        nn.Tanh()
    )

  def forward(self, input):
    output1 = self.conv1(input)

    pool1 = self.maxpool(output1)
    output2 = self.conv2(pool1)
    output3 = self.upsample(output2) + output1
    output4 = self.conv1by1(output3)

    out = self.conv(output4)

    return out

In [13]:
class Discriminator(nn.Module):
  def __init__(self):

    super(Discriminator, self).__init__()

    self.main = nn.Sequential(
        nn.Conv2d(in_channels = 3, out_channels = )
    )